In [1]:
import gc
import json
import pandas as pd
import numpy as np
import seaborn as sns

from tqdm import tqdm_notebook
from glob import glob

from sklearn.decomposition import TruncatedSVD, NMF
from utils import loadpkl, to_json

%matplotlib inline

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [10]:
# load feathers
files = sorted(glob('../features/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm_notebook(files, mininterval=60)], axis=1)

# split train & test
train_df = df[df['click_mode'].notnull()]
test_df = df[df['click_mode'].isnull()]

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [2]:
# load pkls
df = loadpkl('../features/plans.pkl')
queries = loadpkl('../features/queries.pkl')
profiles = loadpkl('../features/profiles.pkl')
queries_pred = loadpkl('../features/queries_pred.pkl')
queries_profiles_pred = loadpkl('../features/queries_profiles_pred.pkl')
queries_plans_pred = loadpkl('../features/queries_plans_pred.pkl')

In [5]:
# merge
df = pd.merge(df, queries, on=['sid','click_mode'], how='left')
df = pd.merge(df, profiles, on='pid', how='left')
df = pd.merge(df, queries_pred, on='sid', how='left')
df = pd.merge(df, queries_profiles_pred, on='sid', how='left')
df = pd.merge(df, queries_plans_pred, on='sid', how='left')

del queries, profiles, queries_pred, queries_profiles_pred, queries_plans_pred
gc.collect()

95

In [2]:
# load csv & pkl
profiles = pd.read_csv('../input/data_set_phase1/profiles.csv')
queries = loadpkl('../features/queries.pkl')

In [3]:
queries

,sid,pid,req_time,click_mode,x_o,y_o,x_d,y_d,queries_o_count,queries_d_count,...,y_d_round,queries_distance_round,queries_x_o_round_count,queries_y_o_round_count,queries_x_d_round_count,queries_y_d_round_count,queries_distance_round_count,queries_o_round_count,queries_d_round_count,queries_o_d_round_count
0,3000821,NaN,2018-11-02 17:54:30,9.0,116.29,39.97,116.32,39.96,1101,1046,...,0,0,164614,164069,155404,159182,182633,62731,59707,21179
1,3085857,210736.0,2018-11-16 10:53:10,1.0,116.39,39.84,116.33,39.79,695,185,...,1,1,256534,58146,155404,49705,221800,21083,14701,1132
2,2944522,NaN,2018-10-06 10:33:58,9.0,116.31,39.93,116.27,40.00,635,4633,...,0,1,164614,286728,155404,159182,221800,56397,59707,5179
3,559931,202427.0,2018-11-23 14:54:11,1.0,116.27,39.88,116.39,39.90,363,8756,...,2,1,164614,286728,261821,301446,221800,56397,173233,15457
4,2819352,172251.0,2018-10-30 11:48:41,7.0,116.34,39.96,116.37,39.86,1166,15545,...,2,1,164614,164069,261821,301446,221800,62731,173233,12497
5,2754575,135606.0,2018-10-18 07:37:32,1.0,116.54,39.73,116.48,39.76,55,75,...,1,1,74642,14988,73005,49705,221800,518,8608,58
6,2224795,189023.0,2018-11-20 14:34:03,1.0,116.34,39.73,116.79,40.35,312,12,...,3,2,164614,14988,1758,7849,855,6080,608,4
7,1598541,156954.0,2018-10-03 09:19:47,1.0,116.26,40.22,116.24,40.22,350,745,...,4,0,164614,17551,35696,15532,182633,5785,7110,649
8,676316,135045.0,2018-11-15 15:25:52,5.0,116.39,39.91,116.39,39.90,3473,8756,...,2,0,256534,286728,261821,301446,182633,153033,173233,74967
9,1068802,176605.0,2018-11-08 23:13:57,3.0,116.29,40.02,116.27,39.95,114,513,...,0,1,164614,164069,155404,159182,221800,62731,59707,21179
